In [10]:
import timm

def print_model_layers(model, prefix=''):
    for name, module in model.named_children():
        if len(list(module.children())) > 0:
            print_model_layers(module, prefix + name + '.')
        else:
            print(f'{prefix +"sep---------------"+ name}: {module}')

# Load the model
model = timm.create_model("hf_hub:timm/maxvit_tiny_tf_224.in1k", pretrained=False)

# Print all layers and sub-modules of the model
print("Printing all layers and sub-modules of the model:")
print_model_layers(model)


Printing all layers and sub-modules of the model:
stem.sep---------------conv1: Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
stem.norm1.sep---------------drop: Identity()
stem.norm1.sep---------------act: GELUTanh()
stem.sep---------------conv2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
stages.0.blocks.0.conv.shortcut.sep---------------pool: AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
stages.0.blocks.0.conv.shortcut.sep---------------expand: Identity()
stages.0.blocks.0.conv.pre_norm.sep---------------drop: Identity()
stages.0.blocks.0.conv.pre_norm.sep---------------act: Identity()
stages.0.blocks.0.conv.sep---------------down: Identity()
stages.0.blocks.0.conv.sep---------------conv1_1x1: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
stages.0.blocks.0.conv.norm1.sep---------------drop: Identity()
stages.0.blocks.0.conv.norm1.sep---------------act: GELUTanh()
stages.0.blocks.0.conv.sep---------------conv2_kxk: Conv2

In [14]:
import timm
import torch
from torch import nn

def print_model_layers(model, prefix=''):
    """
    Prints all layers and sub-modules of a PyTorch model.

    Args:
    - model (torch.nn.Module): The model to inspect.
    - prefix (str): A prefix for layer naming, used for nested models.
    """
    for name, module in model.named_children():
        if len(list(module.children())) > 0:  # If the module has children, recursively print its layers
            print_model_layers(module, prefix + name + '.')
        else:
            print(f'{prefix + name}: {module}')

def get_submodel(model, layer_name):
    """
    Extracts a submodel from the original model up to a specified layer.

    Args:
    - model (torch.nn.Module): The original PyTorch model.
    - layer_name (str): The name of the layer up to which the submodel should be extracted.

    Returns:
    - torch.nn.Module: A submodel that ends at the specified layer.
    """
    submodel = nn.Sequential()
    for name, module in model.named_children():
        submodel.add_module(name, module)
        if name == layer_name:
            break
        # If the current module has children, recurse into it to find the layer
        elif len(list(module.children())) > 0:
            submodel[-1] = get_submodel(module, layer_name)
            if submodel[-1] is not None:
                break
    return submodel

# Load the model
model_name = "hf_hub:timm/maxvit_tiny_tf_224.in1k"
model = timm.create_model(model_name, pretrained=False)

# Print all layers and sub-modules of the model
print("Printing all layers and sub-modules of the model:")
print_model_layers(model)

# Input for layer name
layer_name = input("Enter the layer name up to which you want the submodel: ")

# Extract and print the submodel
submodel = get_submodel(model, layer_name)
if submodel is not None:
    print(f"\nSubmodel up to layer '{layer_name}':")
    print(submodel)
else:
    print(f"Layer '{layer_name}' not found in the model.")


Printing all layers and sub-modules of the model:
stem.conv1: Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
stem.norm1.drop: Identity()
stem.norm1.act: GELUTanh()
stem.conv2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
stages.0.blocks.0.conv.shortcut.pool: AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
stages.0.blocks.0.conv.shortcut.expand: Identity()
stages.0.blocks.0.conv.pre_norm.drop: Identity()
stages.0.blocks.0.conv.pre_norm.act: Identity()
stages.0.blocks.0.conv.down: Identity()
stages.0.blocks.0.conv.conv1_1x1: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
stages.0.blocks.0.conv.norm1.drop: Identity()
stages.0.blocks.0.conv.norm1.act: GELUTanh()
stages.0.blocks.0.conv.conv2_kxk: Conv2dSame(256, 256, kernel_size=(3, 3), stride=(2, 2), groups=256, bias=False)
stages.0.blocks.0.conv.norm2.drop: Identity()
stages.0.blocks.0.conv.norm2.act: GELUTanh()
stages.0.blocks.0.conv.se.fc1: Conv2d(256, 16, kernel_size=(1, 1), s

In [17]:
import timm
import torch
from torch import nn

def print_model_layers(model, prefix=''):
    """
    Prints all layers and sub-modules of a PyTorch model.

    Args:
    - model (torch.nn.Module): The model to inspect.
    - prefix (str): A prefix for layer naming, used for nested models.
    """
    for name, module in model.named_children():
        if len(list(module.children())) > 0:  # If the module has children, recursively print its layers
            print_model_layers(module, prefix + name + '.')
        else:
            print(f'{prefix + name}: {module}')

def extract_submodel(model, layer_name, prefix=''):
    """
    Recursively searches for and extracts a submodel up to the specified layer.

    Args:
    - model (nn.Module): The model to search.
    - layer_name (str): The name of the layer to stop at.
    - prefix (str): Current traversal path, used for matching nested layer names.

    Returns:
    - nn.Sequential: A new model that includes all layers up to the specified one.
    """
    submodel = nn.Sequential()
    found = False
    for name, module in model.named_children():
        current_layer = f"{prefix}{name}"
        if len(list(module.children())) == 0 or current_layer == layer_name:  # Leaf or target layer
            submodel.add_module(name, module)
            if current_layer == layer_name:
                found = True
                break
        else:
            nested_model, nested_found = extract_submodel(module, layer_name, prefix=current_layer + '.')
            if nested_found:
                submodel.add_module(name, nested_model)
                found = True
                break
            else:
                submodel.add_module(name, module)
    return submodel, found

# Load the model
model_name = "hf_hub:timm/maxvit_tiny_tf_224.in1k"
model = timm.create_model(model_name, pretrained=False)

# Print all layers and sub-modules of the model
print("Printing all layers and sub-modules of the model:")
print_model_layers(model)

# Input for layer name
layer_name = input("Enter the layer name up to which you want the submodel: ")

# Extract and print the submodel
submodel, found = extract_submodel(model, layer_name)
if found:
    print(f"\nSubmodel up to layer '{layer_name}':")
    print(submodel)
else:
    print(f"Layer '{layer_name}' not found in the model.")

Printing all layers and sub-modules of the model:
stem.conv1: Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
stem.norm1.drop: Identity()
stem.norm1.act: GELUTanh()
stem.conv2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
stages.0.blocks.0.conv.shortcut.pool: AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
stages.0.blocks.0.conv.shortcut.expand: Identity()
stages.0.blocks.0.conv.pre_norm.drop: Identity()
stages.0.blocks.0.conv.pre_norm.act: Identity()
stages.0.blocks.0.conv.down: Identity()
stages.0.blocks.0.conv.conv1_1x1: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
stages.0.blocks.0.conv.norm1.drop: Identity()
stages.0.blocks.0.conv.norm1.act: GELUTanh()
stages.0.blocks.0.conv.conv2_kxk: Conv2dSame(256, 256, kernel_size=(3, 3), stride=(2, 2), groups=256, bias=False)
stages.0.blocks.0.conv.norm2.drop: Identity()
stages.0.blocks.0.conv.norm2.act: GELUTanh()
stages.0.blocks.0.conv.se.fc1: Conv2d(256, 16, kernel_size=(1, 1), s

In [19]:
import torch
import timm
from torchsummary import summary

# Your existing code for model extraction and printing layers

# Input for layer name
layer_name = input("Enter the layer name up to which you want the submodel: ")

# Extract and print the submodel
submodel, found = extract_submodel(model, layer_name)
if found:
    print(f"\nSubmodel up to layer '{layer_name}':")
    print(submodel)

    # Print summary of the submodel
    summary(submodel, input_size=(3, 224, 224))  # Adjust input size as per your actual input size
else:
    print(f"Layer '{layer_name}' not found in the model.")

KeyboardInterrupt: Interrupted by user

In [25]:
import torch
import timm
from torchsummary import summary

# Function to recursively generate options for selecting the layer
def generate_layer_options(model, prefix=''):
    options = []
    for idx, (name, module) in enumerate(model.named_children(), 1):
        if len(list(module.children())) > 0:  # If the module has children, recursively generate options
            options += generate_layer_options(module, prefix + name + '.')
        else:
            options.append((f'{prefix + name}', module))
    return options

# Load your model here (replace 'model' with your actual model)
model = timm.create_model('resnet50', pretrained=True)

# Generate options for selecting the layer
layer_options = generate_layer_options(model)
print("Options for selecting the layer:")
for idx, (layer_name, _) in enumerate(layer_options, 1):
    print(f"{idx}. {layer_name}")

# Input for selecting the layer
selection = int(input("Enter the number corresponding to the layer you want to select: "))

# Extract and print the submodel based on the selected layer
if 1 <= selection <= len(layer_options):
    selected_layer_name, selected_module = layer_options[selection - 1]
    if hasattr(selected_module, 'in_features'):
        in_features = selected_module.in_features
        print(f"\nThe selected layer '{selected_layer_name}' has {in_features} input features.")
    else:
        print(f"\nThe selected layer '{selected_layer_name}' does not have the 'in_features' attribute.")
else:
    print("Invalid selection. Please select a number within the provided options.")


Options for selecting the layer:
1. conv1
2. bn1
3. act1
4. maxpool
5. layer1.0.conv1
6. layer1.0.bn1
7. layer1.0.act1
8. layer1.0.conv2
9. layer1.0.bn2
10. layer1.0.drop_block
11. layer1.0.act2
12. layer1.0.aa
13. layer1.0.conv3
14. layer1.0.bn3
15. layer1.0.act3
16. layer1.0.downsample.0
17. layer1.0.downsample.1
18. layer1.1.conv1
19. layer1.1.bn1
20. layer1.1.act1
21. layer1.1.conv2
22. layer1.1.bn2
23. layer1.1.drop_block
24. layer1.1.act2
25. layer1.1.aa
26. layer1.1.conv3
27. layer1.1.bn3
28. layer1.1.act3
29. layer1.2.conv1
30. layer1.2.bn1
31. layer1.2.act1
32. layer1.2.conv2
33. layer1.2.bn2
34. layer1.2.drop_block
35. layer1.2.act2
36. layer1.2.aa
37. layer1.2.conv3
38. layer1.2.bn3
39. layer1.2.act3
40. layer2.0.conv1
41. layer2.0.bn1
42. layer2.0.act1
43. layer2.0.conv2
44. layer2.0.bn2
45. layer2.0.drop_block
46. layer2.0.act2
47. layer2.0.aa
48. layer2.0.conv3
49. layer2.0.bn3
50. layer2.0.act3
51. layer2.0.downsample.0
52. layer2.0.downsample.1
53. layer2.1.conv1
54. la